In [5]:
#在这部分在之前的特征部分加入了过去60天的交易行为数据
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.datasets import  make_classification

class Credit_risk_scorePreLR:
    def __init__(self):
        self.data_train_tag, self.data_train_trd = self.load_train_data()
        self.data_test_tag, self.data_test_trd = self.load_test_data()
        self.train_data = self.merge_train_data()
        self.test_data = self.merge_test_data()
        self.train, self.test = self.split_data()

    def load_train_data(self):
        data_tag= pd.read_csv("E:/招行数据比赛/data_b/训练数据集/训练数据集/训练数据集_onehot_tag.csv")
        data_trd = pd.read_csv("E:/招行数据比赛/data_b/训练数据集/训练数据集/训练数据集_trd.csv")
        return data_tag, data_trd
        
    def load_test_data(self):
        data_tag= pd.read_csv("E:/招行数据比赛/data_b/评分数据集b/b/评分数据集_onehot_tag_b.csv")
        data_trd = pd.read_csv("E:/招行数据比赛/data_b/评分数据集b/b/评分数据集_trd_b.csv")
        return data_tag, data_trd
    
    def trd_data_dig_one(self, type):
        if type == 1:
            data_trd = self.data_train_trd
        if type == 2:
            data_trd = self.data_test_trd
        data_trd['trx_tm'] = pd.to_datetime(data_trd['trx_tm'])
        data_trd = data_trd.set_index('trx_tm')
        
        #按照周的时间间隔统计不同的收支交易方向的总金额
        data_trd_group = data_trd.groupby(['id','Dat_Flg1_Cd']).resample('w').sum()#按照'id','Dat_Flg1_Cd'分组，再按照列trx_tm按周重采样，求和
        data_trd_group_weekly_money=data_trd_group.pivot_table(index ='id' , columns= ['trx_tm','Dat_Flg1_Cd'],values='cny_trx_amt' ,aggfunc='sum') .fillna(0)
        if type == 1:
            data_tag = self.data_train_tag
        if type == 2:
            data_tag = self.data_test_tag
            
        data_tag = data_tag.set_index('id')
        data = pd.concat([data_tag, data_trd_group_weekly_money], axis = 1)        #将两个表连接到一起
        #按照周的时间间隔统计不同的收支交易方向的次数
        data_trd_group_times = data_trd.groupby(['id','Dat_Flg1_Cd']).resample('w').count()
        data_trd_group_times.drop(['id','Dat_Flg1_Cd'], axis=1, inplace=True)
        data_trd_group_weekly_times=data_trd_group_times.pivot_table(index ='id' , columns= ['trx_tm','Dat_Flg1_Cd'],values='cny_trx_amt' ,aggfunc='sum') .fillna(0)
        data = pd.concat([data, data_trd_group_weekly_times], axis = 1)
        
        #按照周的时间间隔统计不同的支付方式的总金额
        data_trd_Dat_Flg3_Cd = data_trd.groupby(['id','Dat_Flg3_Cd']).resample('w').sum()
        data_trd_Dat_Flg3_Cd_money=data_trd_Dat_Flg3_Cd.pivot_table (index ='id' , columns= ['trx_tm','Dat_Flg3_Cd'],values='cny_trx_amt' ,aggfunc='sum') .fillna(0)
        data = pd.concat([data, data_trd_Dat_Flg3_Cd_money], axis = 1)
        
        #按照周的时间间隔统计不同的支付方式的次数
        data_trd_Dat_Flg3_Cd_times = data_trd.groupby(['id','Dat_Flg3_Cd']).resample('w').count()
        data_trd_Dat_Flg3_Cd_times.drop(['id','Dat_Flg3_Cd'], axis=1, inplace=True)
        data_trd_group_Dat_Flg3_Cd_times=data_trd_Dat_Flg3_Cd_times.pivot_table (index ='id' , columns= ['trx_tm','Dat_Flg3_Cd'],values='cny_trx_amt' ,aggfunc='sum') .fillna(0)
        data = pd.concat([data, data_trd_group_Dat_Flg3_Cd_times], axis = 1)
        
        #按照周的时间间隔统计不同的收支一级分类的总金额
        data_trd_Trx_Cod1_Cd = data_trd.groupby(['id','Trx_Cod1_Cd']).resample('w').sum()#按照'id','Trx_Cod1_Cd'分组，再按照列trx_tm按周重采样，求和
        data_trd_Trx_Cod1_Cd.drop(['Trx_Cod1_Cd'], axis=1, inplace=True)
        data_trd_Trx_Cod1_Cd_weekly_money=data_trd_Trx_Cod1_Cd.pivot_table(index ='id' , columns= ['trx_tm','Trx_Cod1_Cd'],values='cny_trx_amt' ,aggfunc='sum') .fillna(0)
        data = pd.concat([data, data_trd_Trx_Cod1_Cd_weekly_money], axis = 1)
        
        #按照周的时间间隔统计不同的收支一级分类的次数
        data_trd_Trx_Cod1_Cd_times = data_trd.groupby(['id','Trx_Cod1_Cd']).resample('w').count()#按照'id','Trx_Cod1_Cd'分组，再按照列trx_tm按周重采样，求和
        data_trd_Trx_Cod1_Cd_times.drop(['id','Trx_Cod1_Cd'], axis=1, inplace=True)
        data_trd_Trx_Cod1_Cd_weekly_times=data_trd_Trx_Cod1_Cd_times.pivot_table(index ='id' , columns= ['trx_tm','Trx_Cod1_Cd'],values='cny_trx_amt' ,aggfunc='sum') .fillna(0)
        data = pd.concat([data, data_trd_Trx_Cod1_Cd_weekly_times], axis = 1)
       
#         data.to_csv("E:/招行数据比赛/data_b/训练数据集/训练数据集/训练数据集_trd1.csv", index=True)
        data = data.reset_index() #重置index
        data = data.rename(columns={'index':'id'})
        
        return data

    def trd_data_dig_two(self, type):
        if type == 1:
            data_trd = self.data_train_trd
        if type == 2:
            data_trd = self.data_test_trd
        data_trd['trx_tm'] = pd.to_datetime(data_trd['trx_tm'], format='%Y/%m/%d')  # 转变为8位时间字符
        data_trd['trx_tm'] = data_trd['trx_tm'].dt.strftime('%Y%m%d')
        #收支统计特征：总和、最大、最小、均值、标准差、数目
        temp1 = data_trd[data_trd['cny_trx_amt'] > 0].groupby(by=['id'], as_index=False)['cny_trx_amt'].agg(
        {'income_sum': 'sum', 'income_max': 'max', 'income_min': 'min', 'income_mean': 'mean', 'income_count': 'count',
         'income_std': 'std'}) #收入
        temp2 = data_trd[data_trd['cny_trx_amt'] < 0].groupby(by=['id'], as_index=False)['cny_trx_amt'].agg(  # 支出为负，min、max互换
        {'expend_sum': 'sum', 'expend_max': 'min', 'expend_min': 'max', 'expend_mean': 'mean', 'expend_count': 'count',
         'ecpend_std': 'std'}) #支出
        temp3 = data_trd.groupby(by=['id'], as_index=False)['cny_trx_amt'].agg({'trd_mean': 'mean', 'trd_std': 'std'})
        #收支交易天数
        temp4 = data_trd.groupby(by=['id'], as_index=False)['trx_tm'].agg({'trd_day_count': 'nunique'})  # 交易天数
        temp5 = data_trd[data_trd['cny_trx_amt'] > 0].groupby(by=['id'], as_index=False)['trx_tm'].agg(
        {'income_day_count': 'nunique'})  # 收入天数
        temp6 = data_trd[data_trd['cny_trx_amt'] < 0].groupby(by=['id'], as_index=False)['trx_tm'].agg(
        {'expend_day_count': 'nunique'})  # 支出天数
        data_t = pd.merge(temp3, temp1, on=['id'], how='left')
        data_t = pd.merge(data_t, temp2, on=['id'], how='left')
        data_t = pd.merge(data_t, temp4, on=['id'], how='left')
        data_t = pd.merge(data_t, temp5, on=['id'], how='left')
        data_t = pd.merge(data_t, temp6, on=['id'], how='left')

        # 求总和
        data_t['trd_sum'] = data_t['income_sum'] + data_t['expend_sum']  # 余额
        data_t['trd_total'] = data_t['income_sum'] - data_t['expend_sum']  # 总交易额
        data_t['trd_count'] = data_t['income_count'] + data_t['expend_count']  # 交易总数
        # 收入、支出、交易额求平均
        data_t['trd_every_total'] = data_t['trd_total'] / data_t['trd_count']  # 平均每笔交易额
        data_t['trd_day_total'] = data_t['trd_total'] / data_t['trd_day_count']  # 平均每天交易额
        data_t['income_every_sum'] = data_t['income_sum'] / data_t['income_count']  # 平均每笔收入
        data_t['income_day_sum'] = data_t['income_sum'] / data_t['income_day_count']  # 平均每天收入
        data_t['expend_every_sum'] = data_t['expend_sum'] / data_t['expend_count']  # 平均每笔支出
        data_t['expend_day_sum'] = data_t['expend_sum'] / data_t['expend_day_count']  # 平均每天支出
        # 收入数、支出数、交易数求平均
        data_t['trd_daymean_count'] = data_t['trd_count'] / data_t['trd_day_count']  # 平均每天交易数
        data_t['income_daymean_count'] = data_t['income_count'] / data_t['income_day_count']  # 平均每天收入数
        data_t['expend_daymean_count'] = data_t['expend_count'] / data_t['expend_day_count']  # 平均每天支出数
        # 数量、交易额占比
        data_t['income_count_ratio']=data_t['income_count']/data_t['trd_count']
        data_t['expend_count_ratio']=data_t['expend_count']/data_t['trd_count']
        data_t['income_total_ratio']=data_t['income_sum']/data_t['trd_total']
        data_t['expend_total_ratio']=-data_t['expend_sum']/data_t['trd_total']
        return data_t
    
    def trd_data_dig_three(self, type):
        if type == 1:
            data_trd = self.data_train_trd
        if type == 2:
            data_trd = self.data_test_trd
        data_trd['trx_tm'] = pd.to_datetime(data_trd['trx_tm'], format='%Y/%m/%d')  # 转变为8位时间字符
        data_trd['trx_tm'] = data_trd['trx_tm'].dt.strftime('%Y%m%d')
        data_trd['trx_tm'] = data_trd['trx_tm'].astype(float)#转换成int
        #交易次数特征
        trx_one=data_trd.groupby(['id','trx_tm'])['cny_trx_amt'].count().unstack().reset_index()
        col=trx_one.columns.tolist()[1:]
        trx_one['trdnum_day_max']=trx_one[col].max(axis=1)    #单日最大交易次数
        trx_one=trx_one[['id','trdnum_day_max']]
        #交易额特征---不计正负
        temp=data_trd[['id','trx_tm','cny_trx_amt']].copy()
        temp['cny_trx_amt']=temp['cny_trx_amt'].apply(lambda x:abs(x)) #取绝对值
        trx_two=temp.groupby(['id','trx_tm'])['cny_trx_amt'].sum().unstack().reset_index()
        col=trx_two.columns.tolist()[1:]
        trx_two['trdtotal_day_max']=trx_two[col].max(axis=1)    #单日最大交易额
        trx_two['trdtotal_day_min']=trx_two[col].min(axis=1)    #单日最小交易额
        trx_two=trx_two[['id','trdtotal_day_max','trdtotal_day_min']]
        trx_three=temp.groupby(by=['id'], as_index=False)['cny_trx_amt'].agg(   #单笔最大交易额，单笔最小交易额
            {'every_max':'max','every_min':'min'})
        trx_two=pd.merge(trx_two, trx_three, on=['id'], how='left')

        #余额特征--计算正负值
        trx_four=data_trd.groupby(['id','trx_tm'])['cny_trx_amt'].sum().unstack().reset_index()
        col=trx_four.columns.tolist()[1:]
        trx_four['trdsum_day_max']=trx_four[col].max(axis=1)    #单日最大余额
        trx_four['trdsum_day_min']=trx_four[col].min(axis=1)    #单日最小余额
        trx_four=trx_four[['id','trdsum_day_max','trdsum_day_min']]
        trx_five=data_trd.groupby(by=['id'],as_index=False)['trx_tm'].agg({'trx_date_min':'min',
                                                                  'trx_date_max':'max'})  #交易的第一天和最后一天
        trx_one=pd.merge(trx_one, trx_two, on=['id'], how='left')
        trx_one=pd.merge(trx_one, trx_four, on=['id'], how='left')
        trx_one=pd.merge(trx_one, trx_five, on=['id'], how='left')
        return trx_one
    
    def merge_train_data(self):
        data_one = self.trd_data_dig_one(1)
        data_two = self.trd_data_dig_two(1)
        data_three = self.trd_data_dig_three(1)
        data = pd.merge(data_one, data_two,on=['id'], how='left') 
        data = pd.merge(data, data_three,on=['id'], how='left')
        cname = []
        for i in range(231):
            cname.append(i)
        data.columns = cname
#         data.drop(0 ,axis=1, inplace=True)
        data = data.rename(columns={ 0 :'id'})
        data = data.rename(columns={ 1 :'flag'})
        data.to_csv("E:/招行数据比赛/data_b/训练数据集/训练数据集/训练数据集_data.csv", index=True)
        return data
    
    def merge_test_data(self):
        data_one = self.trd_data_dig_one(2)
        data_two = self.trd_data_dig_two(2)
        data_three = self.trd_data_dig_three(2)
        data = pd.merge(data_one, data_two,on=['id'], how='left') 
        data = pd.merge(data, data_three,on=['id'], how='left') 
        cname = []
        for i in range(230):
            cname.append(i)
        data.columns = cname
#         data.drop(0 ,axis=1, inplace=True)
        data = data.rename(columns={ 0 :'id'})
        data.to_csv("E:/招行数据比赛/data_b/评分数据集b/b/评分数据集_data.csv", index=True)
        return data

    #将数据集拆分成训练集和测试集
    def split_data(self):
        train, test = train_test_split(
        self.train_data,
        test_size=0.1,
        random_state= 40
        )
        return train, test
        
    #模型训练
    def train_model(self):
        label = "flag"
        ID = "id"
        x_columns = [x for x in self.train.columns if x not in [label, ID]]
        x_train = self.train[x_columns]
        y_train = self.train[label]
        x_columns1 = [x for x in self.test.columns if x not in [label, ID]]
        x_test = self.test[x_columns1]
        y_test = self.test[label]
        # 转换为Dataset数据格式
        lgb_train = lgb.Dataset(x_train, y_train)
        lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)
        # 参数
        params = {
            'task': 'train',
            'boosting_type': 'gbdt',  # 设置提升类型
            'objective': 'regression',  # 目标函数
            'metric': {'l2', 'auc'},  # 评估函数
            'num_leaves': 31,  # 叶子节点数
            'learning_rate': 0.05,  # 学习速率
            'feature_fraction': 0.9,  # 建树的特征选择比例
            'bagging_fraction': 0.8,  # 建树的样本采样比例
            'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
            'verbose': 1  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
        }
        # 模型训练
        print("Start Train Model...")
        gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=5)
        print('Save model...')
        # 保存模型到文件
        gbm.save_model('model.txt')
        return gbm

    #模型评估
    def evaluate(self, gbm):
        ID = "id"
        x_columns = [x for x in self.test_data.columns if x not in [ID]]
        x_test = self.test_data[x_columns]
#         lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
        y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)
        print(y_pred)
        f1 = open("E:/招行数据比赛/data_b/评分数据集b/b/提交结果.txt","w")
        for i in y_pred:
            f1.writelines(str(i)+'\n')
        f1.close()
        
if __name__ == "__main__":
    pred = Credit_risk_scorePreLR()
    gbm_model = pred.train_model()
    pred.evaluate(gbm_model)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

D:\Anaconda3\lib\site-packages\ipykernel_launcher

Start Train Model...
[1]	valid_0's auc: 0.700645	valid_0's l2: 0.169602
Training until validation scores don't improve for 5 rounds
[2]	valid_0's auc: 0.718038	valid_0's l2: 0.16791
[3]	valid_0's auc: 0.727962	valid_0's l2: 0.1659
[4]	valid_0's auc: 0.73289	valid_0's l2: 0.16399
[5]	valid_0's auc: 0.737483	valid_0's l2: 0.162253
[6]	valid_0's auc: 0.738996	valid_0's l2: 0.160632
[7]	valid_0's auc: 0.738475	valid_0's l2: 0.159323
[8]	valid_0's auc: 0.738576	valid_0's l2: 0.158253
[9]	valid_0's auc: 0.738614	valid_0's l2: 0.1569
[10]	valid_0's auc: 0.739901	valid_0's l2: 0.155692
[11]	valid_0's auc: 0.742875	valid_0's l2: 0.15455
[12]	valid_0's auc: 0.742809	valid_0's l2: 0.153619
[13]	valid_0's auc: 0.74471	valid_0's l2: 0.152597
[14]	valid_0's auc: 0.744375	valid_0's l2: 0.15163
[15]	valid_0's auc: 0.744992	valid_0's l2: 0.150958
[16]	valid_0's auc: 0.745782	valid_0's l2: 0.150176
[17]	valid_0's auc: 0.746477	valid_0's l2: 0.149453
[18]	valid_0's auc: 0.747023	valid_0's l2: 0.148695
[1